In [27]:
import sys
from timeit import default_timer as timer

import numpy as np
import pandas as pd
import statsmodels.stats.multitest as smm


In [28]:
pValFile = '../output/pValueArraySlow.tsv'

pValDf = pd.read_csv(pValFile, sep='\t')

# Get the first row as df
pValDf = pValDf.iloc[0:1, 1:]
pValDf = pValDf.T
pValDf.columns = ['pVal']
# Remove rows with 0.0 pVal
pValDf = pValDf[pValDf['pVal'] != 0.0]
pValDf.head()


,pVal
ARNT,-0.8222
ATF2,0.0828
ATF4,0.1898
ATF6,0.9257
BCL6,0.9139


In [29]:
#Sort by pVal
pValDf['pVal'] = np.abs(pValDf['pVal'])
pValDf = pValDf.sort_values(by='pVal', ascending=True)
m = len(pValDf)
pValDf['rank'] = np.arange(1, m + 1)
pValDf.head()

,pVal,rank
SP3,0.0126,1
ATF2,0.0828,2
JUN,0.1071,3
ESR1,0.1093,4
MYB,0.1188,5


In [30]:
Q = 0.05

pValDf['fdr'] = (pValDf['rank'] / m) * Q
pValDf.head(10)

,pVal,rank,fdr
SP3,0.0126,1,0.000633
ATF2,0.0828,2,0.001266
JUN,0.1071,3,0.001899
ESR1,0.1093,4,0.002532
MYB,0.1188,5,0.003165
RELA,0.1453,6,0.003797
ATF4,0.1898,7,0.004430
NFKB1,0.2016,8,0.005063
MAML1,0.2104,9,0.005696
STAT3,0.2558,10,0.006329


In [31]:
pValDf.to_csv('../output/pValFdr.tsv', sep='\t')